<a href="https://colab.research.google.com/github/Ugo5738/Kowope_Mart/blob/main/DSN_KOWOPE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install catboost

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
% matplotlib inline


from sklearn.base import BaseEstimator, TransformerMixin


import xgboost
from catboost import CatBoostClassifier
from lightgbm import LGBMModel,LGBMClassifier
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, VotingClassifier
from mlxtend.classifier import StackingClassifier
from sklearn.linear_model import LinearRegression


from sklearn import model_selection
from sklearn.model_selection import train_test_split, RandomizedSearchCV, StratifiedKFold
from sklearn.metrics import roc_auc_score


import os, gc, warnings
warnings.filterwarnings('ignore')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
Train = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Train.csv')
Test = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Test.csv')

In [ ]:
def fill_arbitary(col):
    for i in col:
        b = -999999
        Train[i].fillna(b,inplace=True)
        Test[i].fillna(b,inplace= True)

In [ ]:
def model_predict(estimator,train,label,test, estimator_name):
    mean_train = []
    mean_test_val = []
    test_pred = np.zeros(test.shape[0])
    val_pred = np.zeros(train.shape[0])
    for count, (train_index,test_index) in enumerate(skf.split(train,label)):
        x_train,x_test = train.iloc[train_index],train.iloc[test_index]
        y_train,y_test = label.iloc[train_index],label.iloc[test_index]
        print(f'========================Fold{count +1}==========================')
        estimator.fit(x_train, y_train)
        train_predict = estimator.predict_proba(x_train)[:,1]
        test_predict = estimator.predict_proba(x_test)[:,1]
        val_pred[test_index] = test_predict
        test_pred+= estimator.predict_proba(test)[:,1]
        
        print('\nValidation scores', roc_auc_score(y_test,test_predict))
        print('\nTraining scores', roc_auc_score(y_train,train_predict))
        mean_train.append(roc_auc_score(y_train, train_predict))
        mean_test_val.append(roc_auc_score(y_test,test_predict))
    print('Average Testing ROC score for 10 folds split:',np.mean(mean_test_val))
    print('Average Training ROC score for 10 folds split:',np.mean(mean_train))
    print('standard Deviation for 10 folds split:',np.std(mean_test_val))
    return val_pred, test_pred, estimator_name
        

def lgbm_predict(estimator,train,label,test,estimator_name):
    mean_train = []
    mean_test_val = []
    test_pred = np.zeros(test.shape[0])
    val_pred = np.zeros(train.shape[0])
    for count, (train_index,test_index) in enumerate(skf.split(train,label)):
        x_train,x_test = train.iloc[train_index],train.iloc[test_index]
        y_train,y_test = label.iloc[train_index],label.iloc[test_index]
        print(f'========================Fold{count +1}==========================')
        estimator.fit(x_train,y_train,eval_set=[(x_test,y_test)],early_stopping_rounds=200,
                               verbose=250)
        train_predict = estimator.predict_proba(x_train, num_iteration = estimator.best_iteration_)[:,1]
        test_predict = estimator.predict_proba(x_test, num_iteration = estimator.best_iteration_)[:,1]
        val_pred[test_index] = test_predict
        test_pred+= estimator.predict_proba(test, num_iteration = estimator.best_iteration_)[:,1]
        
        print('\nValidation scores', roc_auc_score(y_test,test_predict))
        print('\nTraining scores', roc_auc_score(y_train,train_predict))
        mean_train.append(roc_auc_score(y_train, train_predict))
        mean_test_val.append(roc_auc_score(y_test,test_predict))
    print('Average Testing ROC score for 10 folds split:',np.mean(mean_test_val))
    print('Average Training ROC score for 10 folds split:',np.mean(mean_train))
    print('standard Deviation for 10 folds split:',np.std(mean_test_val))
    return val_pred, test_pred, estimator_name


def xgb_predict(estimator,train,label,test,estimator_name):
    mean_train = []
    mean_test_val = []
    test_pred = np.zeros(test.shape[0])
    val_pred = np.zeros(train.shape[0])
    for count, (train_index,test_index) in enumerate(skf.split(train,label)):
        x_train,x_test = train.iloc[train_index],train.iloc[test_index]
        y_train,y_test = label.iloc[train_index],label.iloc[test_index]
        print(f'========================Fold{count +1}==========================')
        estimator.fit(x_train, y_train, early_stopping_rounds = 200, eval_metric="auc",
                           eval_set=[(x_test, y_test)],verbose=250)
        train_predict = estimator.predict_proba(x_train, ntree_limit = estimator.get_booster().best_ntree_limit)[:,1]
        test_predict = estimator.predict_proba(x_test, ntree_limit = estimator.get_booster().best_ntree_limit)[:,1]
        val_pred[test_index] = test_predict
        test_pred+= estimator.predict_proba(test, ntree_limit = estimator.get_booster().best_ntree_limit)[:,1]
        
        print('\nTesting scores', roc_auc_score(y_test,test_predict))
        print('\nTraining scores', roc_auc_score(y_train,train_predict))
        mean_train.append(roc_auc_score(y_train, train_predict))
        mean_test_val.append(roc_auc_score(y_test,test_predict))
    print('Average Testing ROC score for 10 folds split:',np.mean(mean_test_val))
    print('Average Training ROC score for 10 folds split:',np.mean(mean_train))
    print('standard Deviation for 10 folds split:',np.std(mean_test_val))
    return val_pred, test_pred, estimator_name

def cat_predict(estimator,train,label,test,estimator_name):
    mean_train = []
    mean_test_val = []
    test_pred = np.zeros(test.shape[0])
    val_pred = np.zeros(train.shape[0])
    for count, (train_index,test_index) in enumerate(skf.split(train,label)):
        x_train,x_test = train.iloc[train_index],train.iloc[test_index]
        y_train,y_test = label.iloc[train_index],label.iloc[test_index]
        print(f'========================Fold{count +1}==========================')
        estimator.fit(x_train,y_train,eval_set=[(x_test,y_test)],early_stopping_rounds=200,
                           verbose=250,use_best_model=True)
        train_predict = estimator.predict_proba(x_train)[:,1]
        test_predict = estimator.predict_proba(x_test)[:,1]
        val_pred[test_index] = test_predict
        test_pred+= estimator.predict_proba(test)[:,1]
        
        print('\nTesting scores', roc_auc_score(y_test,test_predict))
        print('\nTraining scores', roc_auc_score(y_train,train_predict))
        mean_train.append(roc_auc_score(y_train, train_predict))
        mean_test_val.append(roc_auc_score(y_test,test_predict))
    print('Average Testing ROC score for 10 folds split:',np.mean(mean_test_val))
    print('Average Training ROC score for 10 folds split:',np.mean(mean_train))
    print('standard Deviation for 10 folds split:',np.std(mean_test_val))
    return val_pred, test_pred, estimator_name

In [ ]:
class TargetEncoder(BaseEstimator, TransformerMixin):
    """Target encoder.
    
    Replaces categorical column(s) with the mean target value for
    each category.

    """
    
    def __init__(self, cols=None):
        """Target encoder
        
        Parameters
        ----------
        cols : list of str
            Columns to target encode.  Default is to target 
            encode all categorical columns in the DataFrame.
        """
        if isinstance(cols, str):
            self.cols = [cols]
        else:
            self.cols = cols
        
        
    def fit(self, X, y):
        """Fit target encoder to X and y
        
        Parameters
        ----------
        X : pandas DataFrame, shape [n_samples, n_columns]
            DataFrame containing columns to encode
        y : pandas Series, shape = [n_samples]
            Target values.
            
        Returns
        -------
        self : encoder
            Returns self.
        """
        
        # Encode all categorical cols by default
        if self.cols is None:
            self.cols = [col for col in X 
                         if str(X[col].dtype)=='object']

        # Check columns are in X
        for col in self.cols:
            if col not in X:
                raise ValueError('Column \''+col+'\' not in X')

        # Encode each element of each column
        self.maps = dict() #dict to store map for each column
        for col in self.cols:
            tmap = dict()
            uniques = X[col].unique()
            for unique in uniques:
                tmap[unique] = y[X[col]==unique].mean()
            self.maps[col] = tmap
            
        return self

        
    def transform(self, X, y=None):
        """Perform the target encoding transformation.
        
        Parameters
        ----------
        X : pandas DataFrame, shape [n_samples, n_columns]
            DataFrame containing columns to encode
            
        Returns
        -------
        pandas DataFrame
            Input DataFrame with transformed columns
        """
        Xo = X.copy()
        for col, tmap in self.maps.items():
            vals = np.full(X.shape[0], np.nan)
            for val, mean_target in tmap.items():
                vals[X[col]==val] = mean_target
            Xo[col] = vals
        return Xo
            
            
    def fit_transform(self, X, y=None):
        """Fit and transform the data via target encoding.
        
        Parameters
        ----------
        X : pandas DataFrame, shape [n_samples, n_columns]
            DataFrame containing columns to encode
        y : pandas Series, shape = [n_samples]
            Target values (required!).

        Returns
        -------
        pandas DataFrame
            Input DataFrame with transformed columns
        """
        return self.fit(X, y).transform(X, y)

In [ ]:
fill_arbitary(Train.drop(["Applicant_ID","default_status"],axis=1))

In [ ]:
Train.default_status.replace({"yes":1,"no":0},inplace=True)

In [ ]:
encoder = TargetEncoder()
a = pd.DataFrame(Train.form_field47)
b = pd.DataFrame(Test.form_field47)


X_target_encoded = encoder.fit(a,Train["default_status"])
Train = X_target_encoded.transform(Train)
Test = X_target_encoded.transform(Test)

In [ ]:
Train = pd.get_dummies(Train, columns=['form_field47'])
Test = pd.get_dummies(Test, columns=['form_field47'])

In [ ]:
train = Train.drop(["Applicant_ID","default_status"],1)
target = Train["default_status"]

test = Test.drop(["Applicant_ID"],1)

In [ ]:
skf = StratifiedKFold(n_splits = 10,shuffle=True,random_state=199)

XGBOOST

In [ ]:
clf1=xgboost.XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.3, gamma=0.0,
              learning_rate=0.01, max_delta_step=0, max_depth=6,
              min_child_weight=5, missing=None, n_estimators=700, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=40,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=0.7, verbosity=1)

In [ ]:
XGB_train, XGB_test, XGB_name = xgb_predict(clf1, train, target, test,'XGB')

========================Fold1==========================
[0]	validation_0-auc:0.783569
Will train until validation_0-auc hasn't improved in 200 rounds.
[250]	validation_0-auc:0.831124
[500]	validation_0-auc:0.833935
[699]	validation_0-auc:0.834871

Testing scores 0.8348708571412803

Training scores 0.8726294761950564
========================Fold2==========================
[0]	validation_0-auc:0.790504
Will train until validation_0-auc hasn't improved in 200 rounds.
[250]	validation_0-auc:0.833055
[500]	validation_0-auc:0.835176
[699]	validation_0-auc:0.835863

Testing scores 0.8358631028608515

Training scores 0.8723105346951949
========================Fold3==========================
[0]	validation_0-auc:0.790541
Will train until validation_0-auc hasn't improved in 200 rounds.
[250]	validation_0-auc:0.835962
[500]	validation_0-auc:0.839742
[699]	validation_0-auc:0.841372

Testing scores 0.841372282901621

Training scores 0.8717170006294914
========================Fold4==================

CATBOOST

In [ ]:
clf2=CatBoostClassifier(border_count=200, max_depth=7, n_estimators=5000, l2_leaf_reg=10, learning_rate=0.03,
                        bootstrap_type = 'Bernoulli', silent=False, use_best_model=False, eval_metric='AUC', random_seed=34)

In [ ]:
CAT_train, CAT_test, CAT_name = cat_predict(clf2, train, target, test,'CAT')

========================Fold1==========================
0:	test: 0.7900518	best: 0.7900518 (0)	total: 56.8ms	remaining: 4m 43s
250:	test: 0.8335453	best: 0.8335507 (249)	total: 11s	remaining: 3m 27s
500:	test: 0.8353724	best: 0.8353724 (500)	total: 21.7s	remaining: 3m 15s
750:	test: 0.8353617	best: 0.8355416 (608)	total: 32.5s	remaining: 3m 3s
Stopped by overfitting detector  (200 iterations wait)

bestTest = 0.8355416104
bestIteration = 608

Shrink model to first 609 iterations.

Testing scores 0.8355416104184248

Training scores 0.8729225301874605
========================Fold2==========================
0:	test: 0.7961931	best: 0.7961931 (0)	total: 48.7ms	remaining: 4m 3s
250:	test: 0.8353298	best: 0.8353809 (244)	total: 11s	remaining: 3m 28s
500:	test: 0.8370820	best: 0.8371787 (490)	total: 21.9s	remaining: 3m 16s
750:	test: 0.8374007	best: 0.8374126 (676)	total: 32.9s	remaining: 3m 6s
Stopped by overfitting detector  (200 iterations wait)

bestTest = 0.8374512479
bestIteration = 754

LGBM

In [ ]:
clf3=LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=0.7,
               importance_type='split', learning_rate=0.05, max_depth=3,
               metric='auc', min_child_samples=15, min_child_weight=0.001,
               min_split_gain=0.0, n_estimators=5000, n_jobs=-1, num_leaves=300,
               num_threads=15, num_trees=500, objective=None, random_state=29,
               reg_alpha=4, reg_lambda=4, silent=True, subsample=0.7,
               subsample_for_bin=200000, subsample_freq=3)

In [ ]:
LGBM_train, LGBM_test, LGBM_name = lgbm_predict(clf3, train, target, test,'LGBM')

Training until validation scores don't improve for 200 rounds.
[250]	valid_0's auc: 0.832994
[500]	valid_0's auc: 0.834734
Did not meet early stopping. Best iteration is:
[477]	valid_0's auc: 0.834758

Validation scores 0.8347582313017391

Training scores 0.8565784467131087
========================Fold2==========================
Training until validation scores don't improve for 200 rounds.
[250]	valid_0's auc: 0.83602
[500]	valid_0's auc: 0.837135
Did not meet early stopping. Best iteration is:
[485]	valid_0's auc: 0.837203

Validation scores 0.8372030571447642

Training scores 0.8563558260698734
========================Fold3==========================
Training until validation scores don't improve for 200 rounds.
[250]	valid_0's auc: 0.838294
[500]	valid_0's auc: 0.841396
Did not meet early stopping. Best iteration is:
[500]	valid_0's auc: 0.841396

Validation scores 0.8413960843807278

Training scores 0.8560760244973092
========================Fold4==========================
Training

RANDOM FOREST

In [ ]:
clf4=RandomForestClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=10, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=20, min_samples_split=20,
                       min_weight_fraction_leaf=0.0, n_estimators=400,
                       n_jobs=-1, oob_score=False, random_state=6, verbose=0,
                       warm_start=False)

In [ ]:
RF_train, RF_test, RF_name = model_predict(clf4, train, target, test,'RF')

========================Fold1==========================

Validation scores 0.8306279847787816

Training scores 0.8861992803677387
========================Fold2==========================

Validation scores 0.8313701424932463

Training scores 0.8861823952670765
========================Fold3==========================

Validation scores 0.8340843734838415

Training scores 0.8856023701333954
========================Fold4==========================

Validation scores 0.8418490886189433

Training scores 0.8852444792277329
========================Fold5==========================

Validation scores 0.832393951043807

Training scores 0.8858514945519453
========================Fold6==========================

Validation scores 0.8423337337367709

Training scores 0.8851825165239982
========================Fold7==========================

Validation scores 0.8265494027047229

Training scores 0.8858131740036257
========================Fold8==========================

Validation scores 0.82715673105301

GBM

In [ ]:
clf5 = GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.9, loss='deviance', max_depth=2,
                           max_features=1, max_leaf_nodes=2,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=20, min_samples_split=20,
                           min_weight_fraction_leaf=0.1, n_estimators=200,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=67, subsample=0.7, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [ ]:
GBM_train, GBM_test, GBM_name = model_predict(clf5, train, target, test,'GBM')

========================Fold1==========================

Validation scores 0.824439082787581

Training scores 0.8313294087210702
========================Fold2==========================

Validation scores 0.8246105224269438

Training scores 0.8322945970079092
========================Fold3==========================

Validation scores 0.8298233913002835

Training scores 0.8317526974608214
========================Fold4==========================

Validation scores 0.832517529013227

Training scores 0.8308236018217039
========================Fold5==========================

Validation scores 0.8254409870783839

Training scores 0.8325468495978434
========================Fold6==========================

Validation scores 0.8336345265907417

Training scores 0.8316307609821043
========================Fold7==========================

Validation scores 0.8208539108980529

Training scores 0.8329769084183776
========================Fold8==========================

Validation scores 0.817844937677733

In [ ]:
Train_stack3 = pd.DataFrame(XGB_train)
Train_stack3 = pd.concat([Train_stack3,pd.DataFrame(CAT_train),pd.DataFrame(LGBM_train),
                        pd.DataFrame(RF_train),pd.DataFrame(GBM_train)],1)
Test_stack3 = pd.DataFrame(XGB_test)
Test_stack3 = pd.concat([Test_stack3,pd.DataFrame(CAT_test),pd.DataFrame(LGBM_test),
                        pd.DataFrame(RF_test),pd.DataFrame(GBM_test)],1)
Test_stack3.columns=[XGB_name, CAT_name, LGBM_name, RF_name, GBM_name]
Train_stack3.columns=[XGB_name, CAT_name, LGBM_name, RF_name, GBM_name]
Test_stack3 = Test_stack3/10

In [ ]:
Train_stack3

,XGB,CAT,LGBM,RF,GBM
0,0.260084,0.343473,0.311368,0.273966,0.324018
1,0.232360,0.210872,0.185168,0.231427,0.169643
2,0.404768,0.424632,0.435665,0.465043,0.398389
3,0.219375,0.208893,0.174845,0.210187,0.146196
4,0.425473,0.366119,0.460799,0.446903,0.351695
...,...,...,...,...,...
55995,0.010485,0.006508,0.006888,0.011156,0.009713
55996,0.340953,0.236717,0.316187,0.323715,0.481037
55997,0.084003,0.078433,0.079466,0.107394,0.065693
55998,0.682400,0.757734,0.731954,0.561184,0.508509


In [ ]:
Test_stack3

,XGB,CAT,LGBM,RF,GBM
0,0.323384,0.309210,0.328961,0.382646,0.285797
1,0.379874,0.341332,0.377462,0.367112,0.309156
2,0.385731,0.404763,0.380885,0.378076,0.348609
3,0.712136,0.755976,0.759209,0.654173,0.649934
4,0.161127,0.174181,0.152882,0.180647,0.264255
...,...,...,...,...,...
23995,0.642278,0.720047,0.650161,0.571598,0.473513
23996,0.251263,0.277342,0.273476,0.203948,0.206974
23997,0.313686,0.268914,0.268729,0.388459,0.314914
23998,0.515338,0.515777,0.469424,0.543166,0.529415


In [ ]:
meta_estimator = LinearRegression()
final_prediction = meta_estimator.fit(Train_stack3, target).predict(Test_stack3)

In [ ]:
Train_stack3.corr()

,XGB,CAT,LGBM,RF,GBM
XGB,1.000000,0.988135,0.991783,0.984291,0.952192
CAT,0.988135,1.000000,0.988357,0.965316,0.939354
LGBM,0.991783,0.988357,1.000000,0.969843,0.946061
RF,0.984291,0.965316,0.969843,1.000000,0.942105
GBM,0.952192,0.939354,0.946061,0.942105,1.000000


In [ ]:
final_prediction

array([0.31042478, 0.35989204, 0.39944313, ..., 0.27037769, 0.50510483,
       0.20206328])

In [ ]:
# Create a data frame with two columns: Applicant_ID & default_status. default_status contains your predictions
Applicant_ID = np.array(Test['Applicant_ID'])
Solution = pd.DataFrame(final_prediction, Applicant_ID, columns = ["default_status"])
print(Solution)

               default_status
Apcnt_1000032        0.310425
Apcnt_1000048        0.359892
Apcnt_1000052        0.399443
Apcnt_1000076        0.756601
Apcnt_1000080        0.162549
...                       ...
Apcnt_999940         0.706461
Apcnt_999956         0.278454
Apcnt_999976         0.270378
Apcnt_999984         0.505105
Apcnt_999992         0.202063

[24000 rows x 1 columns]


In [ ]:
# Write your solution to a csv file with the name Solution.csv
Solution.to_csv("Zindi Credit Project37.csv", index_label = ["Applicant_ID"])